# Run MSBEGCL on Kaggle

This notebook sets up the environment, compiles the necessary C++ mining tools, prepares the data, runs the mining algorithm to generate bicliques, and finally trains the MSBEGCL recommender system.

In [ ]:
import os, sys, subprocess, time, shutil, struct, re, itertools, math

# --- Configuration ---
repo_url = 'https://github.com/yangzeha/MSBEGCL.git'
repo_dir = 'MSBEGCL'
model_name = 'MSBEGCL'
dataset_name = 'yelp2018'

# 1. Clean and Clone Repository (Silent)
if os.path.exists(repo_dir):
    try:
        shutil.rmtree(repo_dir)
    except Exception as e:
        subprocess.run(['rm', '-rf', repo_dir], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

try:
    subprocess.run(['git', 'clone', '-b', 'master', repo_url], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
except subprocess.CalledProcessError as e:
    sys.exit(1)

# 2. Setup Directories (Silent)
if os.path.basename(os.getcwd()) != repo_dir:
    os.chdir(repo_dir)

# [Robustness Fix]: Auto-detect nested structure
roots = os.listdir('.')
target_structure_found = False
possible_subdirs = ['.', 'MSBEGCL', 'msbegcl', repo_dir]

for d in possible_subdirs:
    if d == '.': path_to_check = '.'
    else:
        path_to_check = d
        if not os.path.exists(d) or not os.path.isdir(d): continue
    contents = os.listdir(path_to_check)
    if 'SELFRec' in contents and 'Similar-Biclique-Idx' in contents:
        if d != '.': os.chdir(d)
        target_structure_found = True
        break

if not target_structure_found:
    found = False
    for root, dirs, files in os.walk('.'):
        if 'SELFRec' in dirs:
            os.chdir(root)
            found = True
            break
    if not found:
        print("CRITICAL ERROR: Could not locate SELFRec directory anywhere.")

selfrec_path = 'SELFRec'
msbe_path = 'Similar-Biclique-Idx'

# 3. Install Dependencies (Silent)
subprocess.run([sys.executable, '-m', 'pip', 'install', 'PyYAML==6.0.2', 'scipy==1.14.1', '-q'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
try:
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'faiss-cpu', '-q'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
except:
    pass

# [CRITICAL FIX]: Ensure util package integrity and alias config
print("--- Fixing Import/Module Errors ---")
util_dir = os.path.join(selfrec_path, 'util')
if os.path.exists(util_dir):
    # Ensure __init__.py exists
    if not os.path.exists(os.path.join(util_dir, '__init__.py')):
        with open(os.path.join(util_dir, '__init__.py'), 'w') as f: f.write('')
    
    # Alias conf.py to config.py to satisfy any legacy/broken imports
    conf_file = os.path.join(util_dir, 'conf.py')
    config_file = os.path.join(util_dir, 'config.py')
    if os.path.exists(conf_file) and not os.path.exists(config_file):
        shutil.copy(conf_file, config_file)
        print("   [Fix] Aliased util/conf.py -> util/config.py")

# 4. Compile C++ Mining Tools (Silent)
sparsez_dir = 'sparsehash'
if not os.path.exists(sparsez_dir):
    subprocess.run(['git', 'clone', 'https://github.com/sparsehash/sparsehash.git'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    cwd_backup = os.getcwd()
    os.chdir(sparsez_dir)
    try:
        subprocess.run(['chmod', '+x', 'configure'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) 
        subprocess.run(['./configure'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        subprocess.run(['make'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception as e:
        pass
    finally:
        os.chdir(cwd_backup)

# Compile msbe
msbe_src = os.path.join(msbe_path, 'main.cpp')
msbe_exe = './msbe'
if not os.path.exists(msbe_src):
    print(f"CRITICAL ERROR: Source file {msbe_src} not found!")
else:
    subprocess.run(['g++', '-w', '-O3', msbe_src, '-o', msbe_exe, '-I', msbe_path, '-I', 'sparsehash/src', '-D_PrintResults_', '-D_CheckResults_'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(['chmod', '+x', msbe_exe], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 5. Data Preprocessing
print(f'\n--- Preprocessing {dataset_name} for Mining ---')
train_file = os.path.join(selfrec_path, 'dataset', dataset_name, 'train.txt')
mining_graph_txt = 'graph.txt'

if not os.path.exists(train_file):
    print(f"CRITICAL ERROR: Data file {train_file} not found!")
else:
    users = set()
    items = set()
    edges = []
    with open(train_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 2:
                u, i = parts[0], parts[1]
                users.add(u)
                items.add(i)
                edges.append((u, i))

    try:
        sorted_users = sorted(list(users), key=lambda x: int(x))
        sorted_items = sorted(list(items), key=lambda x: int(x))
    except:
        sorted_users = sorted(list(users))
        sorted_items = sorted(list(items))

    u_map = {u: idx for idx, u in enumerate(sorted_users)}
    i_map = {i: idx for idx, i in enumerate(sorted_items)}

    n1 = len(users)
    n2 = len(items)
   
    print(f'Preprocessing graph with {n1} users, {n2} items, {len(edges)} edges.')
    
    total_nodes = n1 + n2
    adj = [[] for _ in range(total_nodes)]
    edge_count = 0
    
    for u, i in edges:
        uid = u_map[u]
        iid = i_map[i] + n1
        adj[uid].append(iid)
        adj[iid].append(uid)
        edge_count += 2
        
    for k in range(total_nodes):
        adj[k].sort()
        
    degree_file = 'graph_b_degree.bin'
    with open(degree_file, 'wb') as f:
        f.write(struct.pack('I', 4))
        f.write(struct.pack('I', n1))
        f.write(struct.pack('I', n2))
        f.write(struct.pack('I', edge_count))
        degrees = [len(adj[k]) for k in range(total_nodes)]
        f.write(struct.pack(f'{total_nodes}I', *degrees))
        
    adj_file = 'graph_b_adj.bin'
    with open(adj_file, 'wb') as f:
        flat_adj = []
        for k in range(total_nodes):
            flat_adj.extend(adj[k])
        f.write(struct.pack(f'{edge_count}I', *flat_adj))
        
    print(f"Generated binary graph files.")
    
    with open(mining_graph_txt, 'w') as f:
        f.write("dummy")

# 6. Run Mining
print('\n--- Mining Bicliques (Structure Discovery) ---')
# [Strategy]: Use relaxed threshold to ensure high recall of structural candidates
sim_threshold = 0.2
size_threshold = 2

if os.path.exists(msbe_exe) and os.path.exists(mining_graph_txt):
    print('Building Index...')
    subprocess.run([msbe_exe, mining_graph_txt, '1', '1', str(sim_threshold), 'GRL3'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    print('Enumerating...')
    raw_bicliques_file = 'bicliques_raw.txt'
    with open(raw_bicliques_file, 'w') as outfile:
        subprocess.run([
            msbe_exe, mining_graph_txt, 
            '0', '1', str(sim_threshold), 'GRL3', 
            '1', 'GRL3', 
            '0', '0', 'heu', 
            '4', str(sim_threshold), str(size_threshold), '2'
        ], stdout=outfile, stderr=subprocess.DEVNULL, check=True)
    
else:
    print("Skipping mining due to compliation or data failure.")

# 7. Process Bicliques -> Model Format
print('\n--- Formatting Bicliques for Model ---')
final_biclique_path = os.path.join(selfrec_path, 'dataset', dataset_name, 'bicliques.txt')
count = 0

if os.path.exists(raw_bicliques_file):
    with open(raw_bicliques_file, 'r') as fr, open(final_biclique_path, 'w') as fw:
        for line in fr:
            line = line.strip()
            if not line: continue
            clean_line = line.replace('|', ' ').replace(',', ' ').replace(':', ' ')
            tokens = clean_line.split()
            current_users = []
            current_items = []
            for t in tokens:
                if not t.isdigit(): continue
                nid = int(t)
                if nid < n1:
                    if nid < len(sorted_users):
                        current_users.append(sorted_users[nid])
                else:
                    iid = nid - n1
                    if iid >= 0 and iid < len(sorted_items):
                        current_items.append(sorted_items[iid])
            if len(current_users) > 0 and len(current_items) > 0:
                fw.write(f"{' '.join(current_users)} | {' '.join(current_items)}\n")
                count += 1
    print(f"Processed {count} bicliques into {final_biclique_path}")
    
# [Patching] Write Helper Files
print('--- Patching Model Code in Notebook ---')
denoising_code = r'''
import numpy as np
import random
from collections import defaultdict

def filter_large_bicliques(bicliques, max_size=30):
    filtered = []
    for users, items in bicliques:
        if len(users) <= max_size and len(items) <= max_size:
            filtered.append((users, items))
    return filtered

def compute_jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0

def prune_low_similarity_clusters(bicliques, interaction_mat, min_sim=0.2):
    pruned = []
    for users, items in bicliques:
        user_sims = []
        user_list = list(users)
        for i in range(len(user_list)):
            for j in range(i+1, len(user_list)):
                try:
                    u1_idx = interaction_mat[user_list[i]].indices
                    u2_idx = interaction_mat[user_list[j]].indices
                except:
                    try:
                         u1_idx = np.where(interaction_mat[user_list[i]] > 0)[0]
                         u2_idx = np.where(interaction_mat[user_list[j]] > 0)[0]
                    except:
                        continue
                u1_interactions = set(u1_idx)
                u2_interactions = set(u2_idx)
                sim = compute_jaccard_similarity(u1_interactions, u2_interactions)
                user_sims.append(sim)
        avg_sim = np.mean(user_sims) if user_sims else 0
        if avg_sim >= min_sim:
            pruned.append((users, items))
    return pruned

def build_enhanced_neighbor_dict(bicliques, user_map, item_map, top_k=10):
    user_neighbors = defaultdict(list)
    item_neighbors = defaultdict(list)
    for users, items in bicliques:
        user_ids = [user_map[u] for u in users if u in user_map]
        item_ids = [item_map[i] for i in items if i in item_map]
        for u in user_ids:
            for other_u in user_ids:
                if u != other_u: user_neighbors[u].append(other_u)
        for i in item_ids:
            for other_i in item_ids:
                if i != other_i: item_neighbors[i].append(other_i)
    enhanced_user = {}
    enhanced_item = {}
    for u, neighbors in user_neighbors.items():
        unique_neighbors = list(set(neighbors))
        if len(unique_neighbors) > top_k:
            enhanced_user[u] = random.sample(unique_neighbors, top_k)
        else:
            enhanced_user[u] = unique_neighbors
    for i, neighbors in item_neighbors.items():
        unique_neighbors = list(set(neighbors))
        if len(unique_neighbors) > top_k:
            enhanced_item[i] = random.sample(unique_neighbors, top_k)
        else:
            enhanced_item[i] = unique_neighbors
    return enhanced_user, enhanced_item
'''
denoise_path = os.path.join(selfrec_path, 'util', 'denoising_helper.py')
os.makedirs(os.path.dirname(denoise_path), exist_ok=True)
with open(denoise_path, 'w') as f:
    f.write(denoising_code)

msbe_helper_code = r'''
import os
import random
import numpy as np
from util.denoising_helper import filter_large_bicliques, prune_low_similarity_clusters, build_enhanced_neighbor_dict

def load_msbe_neighbors(file_path, user_map, item_map, interaction_mat=None, sim_threshold=0.2):
    user_neighbors = {}
    item_neighbors = {}
    print(f"Loading bicliques from {file_path}, sim_threshold={sim_threshold}")
    if os.path.exists(file_path):
        bicliques = []
        with open(file_path, 'r') as f:
            for line in f:
                parts = line.strip().split('|')
                if len(parts) < 2: continue
                users = [u.strip() for u in parts[0].split() if u.strip() in user_map]
                items = [i.strip() for i in parts[1].split() if i.strip() in item_map]
                if len(users) > 0 and len(items) > 0:
                    bicliques.append((users, items))
        print(f"Original bicliques: {len(bicliques)}")
        filtered = filter_large_bicliques(bicliques, max_size=30)
        if interaction_mat is not None:
            try:
                pruned = prune_low_similarity_clusters(filtered, interaction_mat, min_sim=sim_threshold)
            except Exception as e:
                print(f"Warning: Pruning failed ({e}), using filtered only.")
                pruned = filtered
            print(f"Pruned bicliques: {len(pruned)}")
        else:
            pruned = filtered
        user_neighbors, item_neighbors = build_enhanced_neighbor_dict(pruned, user_map, item_map, top_k=15)
        u_c = sum(1 for v in user_neighbors.values() if len(v) > 0)
        i_c = sum(1 for v in item_neighbors.values() if len(v) > 0)
        print(f"Enhanced Neighbors: {u_c} users, {i_c} items.")
    return user_neighbors, item_neighbors
'''
helper_path = os.path.join(selfrec_path, 'util', 'msbe_helper.py')
os.makedirs(os.path.dirname(helper_path), exist_ok=True)
with open(helper_path, 'w') as f:
    f.write(msbe_helper_code)


# 3. MSBEGCL.py (Rocket Strategy + Innovation 3: Hard Negative Mining)
# [Robustness Fix]: Added device agnostic code (CPU/CUDA check) to prevent crashes on non-GPU envs
# [Hotfix]: Added missing 'import os'
model_code = r'''
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from base.graph_recommender import GraphRecommender
from util.sampler import next_batch_pairwise
from base.torch_interface import TorchGraphInterface
from util.loss_torch import bpr_loss, l2_reg_loss
import random
import numpy as np
import traceback
from util.msbe_helper import load_msbe_neighbors

class MSBEGCL(GraphRecommender):
    def __init__(self, conf, training_set, test_set):
        super(MSBEGCL, self).__init__(conf, training_set, test_set)
        
        # [Device Safe]
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # [Robustness Fix]: Use safe check instead of 'in' operator to avoid ModelConf __getitem__ iteration bug
        if not self.config.contain('MSBEGCL'):
            print("WARNING: MSBEGCL config section not found! Using hardcoded defaults.")
            args = {'n_layer': 2, 'lambda': 0.5, 'gamma': 0.05, 'eps': 0.1, 'tau': 0.2, 'biclique.file': ''}
        else:
            args = self.config['MSBEGCL']
            
        self.n_layers = int(args.get('n_layer', 2))
        self.eps = float(args.get('eps', 0.1))
        self.tau = float(args.get('tau', 0.2))
        
        self.cl_rate = float(args.get('lambda', 0.5))
        self.msbe_rate = float(args.get('gamma', 0.05))
        
        # [Innovation 3]: Hard Negative Weight
        self.hard_weight = float(args.get('hard_weight', 2.0))
        
        print(f"MSBEGCL Init: lambda={self.cl_rate}, gamma={self.msbe_rate}, tau={self.tau}, hard_w={self.hard_weight}, device={self.device}")
        
        self.model = MSBEGCL_Encoder(self.data, self.emb_size, self.eps, self.n_layers, self.device)
        
        self.biclique_file = args.get('biclique.file', '')
        self.sim_threshold = float(args.get('sim_threshold', 0.2))
        
        if os.path.exists(self.biclique_file):
            self.user_msb, self.item_msb = self.load_denoised_neighbors()
        else:
            print(f"Warning: Biclique file {self.biclique_file} not found. Running w/o structure.")
            self.user_msb, self.item_msb = {}, {}
        
    def load_denoised_neighbors(self):
        user_neighbors, item_neighbors = load_msbe_neighbors(
            self.biclique_file,
            self.data.user,
            self.data.item,
            self.data.interaction_mat, 
            self.sim_threshold
        )
        return user_neighbors, item_neighbors

    # [Innovation 3]: Structure-Aware Hard Negative InfoNCE
    def info_nce_with_hard_neg(self, view1, view2, nodes, neighbor_dict, full_embeddings):
        """
        Structure-Aware Hard Negative Mining:
        Includes structurally similar nodes (from Bicliques) as heavily weighted negatives
        in the contrastive loss denominator.
        """
        # Normalize inputs for Cosine Similarity
        view1 = F.normalize(view1, dim=1)
        view2 = F.normalize(view2, dim=1)
        
        # 1. Positive Pairs (Diagonal)
        pos_score = torch.sum(view1 * view2, dim=1)
        exp_pos = torch.exp(pos_score / self.tau)
        
        # 2. Standard Batch Negatives (SimGCL default)
        # Similarity between view1 and ALL view2s in batch
        sim_all = torch.matmul(view1, view2.t())
        exp_all = torch.exp(sim_all / self.tau).sum(dim=1)
        
        # 3. Structure-Aware Hard Negatives
        # For each node, pick a 'Hard Negative' from its biclique neighbors
        if self.hard_weight > 0.0:
            nodes_np = nodes.cpu().numpy()
            hard_indices = []
            mask_list = []
            
            for nid in nodes_np:
                neighbors = neighbor_dict.get(nid, [])
                if neighbors:
                    # Pick random structural neighbor as hard negative
                    hard_idx = random.choice(neighbors)
                    hard_indices.append(hard_idx)
                    mask_list.append(1.0)
                else:
                    hard_indices.append(nid) # Dummy (masked out)
                    mask_list.append(0.0)
            
            hard_tensor = torch.tensor(hard_indices, device=view1.device, dtype=torch.long)
            mask = torch.tensor(mask_list, device=view1.device)
            
            # Lookup embedding for hard negative (using main view approximation)
            hard_embs = full_embeddings[hard_tensor]
            hard_embs = F.normalize(hard_embs, dim=1)
            
            # Compute similarity with hard negative
            hard_score = torch.sum(view1 * hard_embs, dim=1)
            exp_hard = torch.exp(hard_score / self.tau)
            
            # Add to denominator with extra weight
            denominator = exp_all + (self.hard_weight * exp_hard * mask)
        else:
            denominator = exp_all
            
        loss = -torch.log(exp_pos / denominator)
        return loss.mean()

    def cal_msbe_loss(self, nodes, neighbors_dict, embeddings):
        """
        Parallel Dual-View Injection (Innovation 1/2)
        """
        if len(nodes) == 0: return torch.tensor(0.0, device=embeddings.device)
        
        nodes_list = nodes.cpu().tolist()
        struct_targets = []
        valid_sources = []
        
        for node in nodes_list:
            if node in neighbors_dict and len(neighbors_dict[node]) > 1:
                m_list = neighbors_dict[node]
                sample_size = min(len(m_list), 10)
                sampled = random.sample(m_list, sample_size)
                
                neighbor_embs = embeddings[sampled]
                center = torch.mean(neighbor_embs.detach(), dim=0)
                
                struct_targets.append(center)
                valid_sources.append(embeddings[node])

        if len(valid_sources) == 0:
            return torch.tensor(0.0, device=embeddings.device)
            
        sources = torch.stack(valid_sources)
        targets = torch.stack(struct_targets)
        
        # Standard InfoNCE (or MSE) for Injection, keep it sharp
        sources = F.normalize(sources, dim=1)
        targets = F.normalize(targets, dim=1)
        sim = torch.sum(sources * targets, dim=1)
        return -torch.log(torch.exp(sim / 0.1)).mean()

    def train(self):
        model = self.model.to(self.device)
        optimizer = torch.optim.Adam(model.parameters(), lr=self.lRate)
        
        for epoch in range(self.maxEpoch):
            try:
                current_gamma = self.msbe_rate 
                
                epoch_rec = 0
                epoch_cl = 0
                epoch_msb = 0
                batch_c = 0
                
                for n, batch in enumerate(next_batch_pairwise(self.data, self.batch_size)):
                    user_idx, pos_idx, neg_idx = batch
                    
                    # Augmentation Views
                    u_v1, i_v1 = model(perturbed=True)
                    u_v2, i_v2 = model(perturbed=True)
                    
                    # Main View (Shared by Rec, MSBE, and Hard Neg Lookup)
                    res_u, res_i = model(perturbed=False)
                    
                    # 1. Rec Loss
                    l_rec = bpr_loss(res_u[user_idx], res_i[pos_idx], res_i[neg_idx]) + \
                            l2_reg_loss(self.reg, res_u[user_idx], res_i[pos_idx], res_i[neg_idx])
                    
                    # 2. Structure-Aware Contrastive Loss (Innovation 3)
                    u_uniq = torch.unique(torch.tensor(user_idx).to(self.device))
                    i_uniq = torch.unique(torch.tensor(pos_idx).to(self.device))
                    
                    # Pass main embeddings (res_u, res_i) to lookup hard negatives
                    cl_u = self.info_nce_with_hard_neg(u_v1[u_uniq], u_v2[u_uniq], u_uniq, self.user_msb, res_u)
                    cl_i = self.info_nce_with_hard_neg(i_v1[i_uniq], i_v2[i_uniq], i_uniq, self.item_msb, res_i)
                    
                    l_sim = self.cl_rate * (cl_u + cl_i)
                                
                    # 3. Parallel Injection Loss (Innovation 1/2)
                    l_msbe = current_gamma * (
                        self.cal_msbe_loss(u_uniq, self.user_msb, res_u) +
                        self.cal_msbe_loss(i_uniq, self.item_msb, res_i)
                    )
                    
                    total_loss = l_rec + l_sim + l_msbe
                    
                    optimizer.zero_grad()
                    total_loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                    
                    epoch_rec += l_rec.item()
                    epoch_cl += l_sim.item()
                    epoch_msb += l_msbe.item()
                    batch_c += 1
                
                print(f'Epoch {epoch} Avg: Rec={epoch_rec/batch_c:.4f} Sim={epoch_cl/batch_c:.4f} Msb={epoch_msb/batch_c:.4f}')
                
                with torch.no_grad():
                    self.user_emb, self.item_emb = self.model(perturbed=False)
                self.fast_evaluation(epoch)

            except Exception as e:
                traceback.print_exc()
                break

    def save(self):
        with torch.no_grad():
            self.best_user_emb, self.best_item_emb = self.model(perturbed=False)

    def predict(self, u):
        u = self.data.get_user_id(u)
        score = torch.matmul(self.user_emb[u], self.item_emb.transpose(0, 1))
        return score.cpu().numpy()

class MSBEGCL_Encoder(nn.Module):
    def __init__(self, data, emb_size, eps, n_layers, device):
        super(MSBEGCL_Encoder, self).__init__()
        self.data = data
        self.eps = eps
        self.emb_size = emb_size
        self.n_layers = n_layers
        self.device = device
        self.sparse_norm_adj = TorchGraphInterface.convert_sparse_mat_to_tensor(data.norm_adj).to(self.device)
        self.embedding_dict = self._init_model()

    def _init_model(self):
        # Using Xavier Init for stability
        initializer = nn.init.xavier_uniform_
        embedding_dict = nn.ParameterDict({
            'user_emb': nn.Parameter(initializer(torch.empty(self.data.user_num, self.emb_size))),
            'item_emb': nn.Parameter(initializer(torch.empty(self.data.item_num, self.emb_size))),
        })
        return embedding_dict

    def forward(self, perturbed=False):
        ego_embeddings = torch.cat([self.embedding_dict['user_emb'], self.embedding_dict['item_emb']], 0)
        all_embeddings = []
        for k in range(self.n_layers):
            ego_embeddings = torch.sparse.mm(self.sparse_norm_adj, ego_embeddings)
            if perturbed:
                random_noise = torch.rand_like(ego_embeddings).to(self.device)
                ego_embeddings += torch.sign(ego_embeddings) * F.normalize(random_noise, dim=-1) * self.eps
            all_embeddings.append(ego_embeddings)
        all_embeddings = torch.stack(all_embeddings, dim=1)
        all_embeddings = torch.mean(all_embeddings, dim=1)
        user_all_embeddings, item_all_embeddings = torch.split(all_embeddings, [self.data.user_num, self.data.item_num])
        return user_all_embeddings, item_all_embeddings
'''
model_path = os.path.join(selfrec_path, 'model', 'graph', 'MSBEGCL.py')
os.makedirs(os.path.dirname(model_path), exist_ok=True)
with open(model_path, 'w') as f:
    f.write(model_code)
print("Patched model/graph/MSBEGCL.py with Structure-Aware Hard Negative Mining")

# --- 8. Auto-Tuner Loop ---
print('\n>>> STARTING AUTO-TUNER SEARCH (Target: Epoch 0/1 Recall@20 > 0.065) <<<')

# Search Space
lambdas = [0.1, 0.2, 0.5]
gammas = [0.01, 0.05, 0.1, 0.2]
taus = [0.1, 0.2]

results_log = []
found_target = False

os.chdir(selfrec_path)
os.makedirs('results', exist_ok=True) # Ensure output dir exists

for lam, gam, tau in itertools.product(lambdas, gammas, taus):
    print(f"\n──────────────────────────────────────────────")
    print(f"Testing Config: lambda={lam}, gamma={gam}, tau={tau}")
    print(f"──────────────────────────────────────────────")
    
    # 1. Generate Config
    clean_biclique_path = f'./dataset/{dataset_name}/bicliques.txt'
    yaml_content = f"""
training.set: ./dataset/{dataset_name}/train.txt
test.set: ./dataset/{dataset_name}/test.txt
output: ./results

model:
  name: MSBEGCL
  type: graph

item.ranking.topN: [10,20]
embedding.size: 64
max.epoch: 2
batch.size: 4096
learning.rate: 0.001
reg.lambda: 0.0001
output: ./results

MSBEGCL:
  n_layer: 2
  lambda: {lam}
  gamma: {gam}
  eps: 0.1
  tau: {tau}
  hard_weight: 2.0  # [Innovation 3]: Enable Hard Negative Mining
  biclique.file: {clean_biclique_path}
  sim_threshold: {sim_threshold}
"""
    # Write config
    with open('conf/MSBEGCL.yaml', 'w') as f:
        f.write(yaml_content)
        
    # 2. Run Process
    process = subprocess.Popen(
        [sys.executable, '-u', 'main.py'],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT, # Merge stderr into stdout
        text=True,
        bufsize=1
    )
    
    # Send Model Name input
    try:
        process.stdin.write('MSBEGCL\n')
        process.stdin.flush()
        process.stdin.close()
    except Exception: pass
    
    # 3. Monitor Output
    run_recall = 0.0
    start_time = time.time()
    line_count = 0
    
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None: break
        
        if line:
            l = line.strip()
            line_count += 1
            # ALWAYS PRINT FIRST 50 LINES for debugging
            if line_count < 50:
                 print(f"   [Debug]: {l}")
            
            # Log interesting lines
            if "Recall" in l or "Traceback" in l or "Error" in l or "Epoch" in l:
                if line_count >= 50: # Avoid dupes
                     print(f"   [Log]: {l[:100]}...") # truncate for cleanliness
                
            # Parse Recall
            # Format usually: "Performance: Recall@[10, 20]: [0.xxx, 0.xxx]" or similar
            if "Recall" in l:
                try:
                    # Find all floats
                    nums = [float(x) for x in re.findall(r"0\.\d+", l)]
                    if nums:
                        # Assuming the metrics are rising or the last ones are @20
                        # Usually Recall@10, Recall@20. So max is likely @20
                        current_max = max(nums)
                        run_recall = max(run_recall, current_max)
                except: pass
                
        # Timeout safety (2 minutes per run is enough for 1 epoch)
        if time.time() - start_time > 180:
            process.kill()
            print("   [Timeout] Killed process.")
            break
            
    # Check Result
    if run_recall > 0.065:
        print(f"   [!!!] SUCCESS! Params ({lam}, {gam}, {tau}) -> Recall {run_recall}")
        results_log.append({
            'lambda': lam,
            'gamma': gam,
            'tau': tau,
            'recall': run_recall
        })
        found_target = True
    else:
        print(f"   [x] Failed. Best Recall: {run_recall}")

print("\n\n========================================")
print("       AUTO-TUNER FINAL REPORT          ")
print("========================================")

if not results_log:
    print("No parameters achieved > 0.065 recall in early epochs.")
else:
    print(f"Found {len(results_log)} successful configurations:\n")
    for res in results_log:
        print(f"Params: lambda={res['lambda']}, gamma={res['gamma']}, tau={res['tau']}")
        print(f"Result: Recall@20 = {res['recall']}")
        print("----------------------------------------")